In [7]:
#This is for aggregated transtaction state wise data
import pandas as pd
import json
import os

#This is to direct the path to get the data as states

path=r"D:/BONEYS/WEB/PYTHON/Project/PhonePe/data/aggregated/user/country/india/state"
Agg_user_state_list=os.listdir(path)


#This is to extract the data's to create a dataframe
clm={'State':[],'Year':[],'Quater':[],'Mobile_Brand_Used':[],'Usage_count':[],'Percentage':[]}

for i in Agg_user_state_list:
    p_i=path+"/"+i
    Agg_yr=os.listdir(p_i)
    #delhi
    for j in Agg_yr:
        p_j=p_i+"/"+j
        #2018
        Agg_yr_list=os.listdir(p_j)
        for k in Agg_yr_list:
            p_k=p_j+"/"+k 
            #1.json
            Data=open(p_k,'r')
            D=json.load(Data)
            #in this, usersByDevice has null value. So checking them
            users = D.get('data', {}).get('usersByDevice')
            if users is None:
                continue
            else:
                for z in users:
                    Brand=z['brand']
                    count=z['count']
                    perc=z['percentage']
                    clm['Mobile_Brand_Used'].append(Brand)
                    clm['Usage_count'].append(count)
                    clm['Percentage'].append(perc)
                    clm['State'].append(i)
                    clm['Year'].append(j)
                    clm['Quater'].append(int(k.strip('.json')))
#Succesfully created a dataframe
Agg_User_state=pd.DataFrame(clm)
print(Agg_User_state)

                          State  Year  Quater Mobile_Brand_Used  Usage_count  \
0     andaman-&-nicobar-islands  2018       1            Xiaomi         1665   
1     andaman-&-nicobar-islands  2018       1           Samsung         1445   
2     andaman-&-nicobar-islands  2018       1              Vivo          982   
3     andaman-&-nicobar-islands  2018       1              Oppo          501   
4     andaman-&-nicobar-islands  2018       1           OnePlus          332   
...                         ...   ...     ...               ...          ...   
6848                west-bengal  2022       1            Lenovo       330017   
6849                west-bengal  2022       1           Infinix       284678   
6850                west-bengal  2022       1              Asus       280347   
6851                west-bengal  2022       1             Apple       277752   
6852                west-bengal  2022       1            Others      2196334   

      Percentage  
0       0.247033  
1

In [5]:
#You need to map your State names to the names used in the GeoJSON file (properties.ST_NM). Otherwise you will get blank figure.
state_name_map = {
    'andaman-&-nicobar-islands': 'Andaman & Nicobar Island',
    'andhra-pradesh': 'Andhra Pradesh',
    'arunachal-pradesh': 'Arunanchal Pradesh',
    'assam': 'Assam',
    'bihar': 'Bihar',
    'chandigarh': 'Chandigarh',
    'chhattisgarh': 'Chhattisgarh',
    'dadra-&-nagar-haveli-&-daman-&-diu': 'Dadra and Nagar Haveli',
    'delhi': 'NCT of Delhi',
    'goa': 'Goa',
    'gujarat': 'Gujarat',
    'haryana': 'Haryana',
    'himachal-pradesh': 'Himachal Pradesh',
    'jammu-&-kashmir': 'Jammu & Kashmir',
    'jharkhand': 'Jharkhand',
    'karnataka': 'Karnataka',
    'kerala': 'Kerala',
    'ladakh': 'Ladakh',
    'lakshadweep': 'Lakshadweep',
    'madhya-pradesh': 'Madhya Pradesh',
    'maharashtra': 'Maharashtra',
    'manipur': 'Manipur',
    'meghalaya': 'Meghalaya',
    'mizoram': 'Mizoram',
    'nagaland': 'Nagaland',
    'odisha': 'Odisha',
    'puducherry': 'Puducherry',
    'punjab': 'Punjab',
    'rajasthan': 'Rajasthan',
    'sikkim': 'Sikkim',
    'tamil-nadu': 'Tamil Nadu',
    'telangana': 'Telangana',
    'tripura': 'Tripura',
    'uttar-pradesh': 'Uttar Pradesh',
    'uttarakhand': 'Uttarakhand',
    'west-bengal': 'West Bengal'
}
Agg_User_state['State'] = Agg_User_state['State'].str.strip().str.lower()
Agg_User_state['State'] = Agg_User_state['State'].replace(state_name_map)

In [ ]:
#creating table to store the above data
import mysql.connector
connection = mysql.connector.connect(
    host="localhost",
    user="xxx", #give your user name. Mostly its root
    password="xxx", #give your password
    database="xxx" #give any database name you want
)
cursor=connection.cursor()
query1 = """
        create table Aggregated_User_State(
        State varchar(100),
        Year int,
        Quater int,
        Mobile_Brand_Used varchar(100),
        Usage_count float,
        Percentage float)
        """
cursor.execute(query1)

In [4]:
#storing the above in mysql database
#all data must be tuple and must be a list of tuples
data=[]
for index in Agg_User_state.index:
    row = Agg_User_state.loc[index]
    row = row.values
    row = row[0],float(row[1]),int(row[2]),row[3],int(row[4]),float(row[5])
    data.append(row)
query2 = "insert into Aggregated_User_State values (%s,%s,%s,%s,%s,%s)"
cursor.executemany(query2,data)
connection.commit()

In [8]:
#doing data cleaning for Agg_User_state
from sklearn.preprocessing import LabelEncoder
#first step check and remove null values
Agg_User_state.isna().sum()
Agg_User_state.info()
#Second do encoding and dummies to make all values int
enc_col = ['Mobile_Brand_Used']
dummy_col = ['Year','State','Quater']
encoder = LabelEncoder()
Agg_User_state['encoded_Mobile_Brand_Used'] = encoder.fit_transform(Agg_User_state['Mobile_Brand_Used'])
Agg_User_state = pd.get_dummies(Agg_User_state,columns=dummy_col,dtype=int)
#third removing outliners
Agg_User_state['registered_Usage_count_std']=(Agg_User_state['Usage_count']-Agg_User_state['Usage_count'].mean())/Agg_User_state['Usage_count'].std()
condition1 = Agg_User_state['registered_Usage_count_std']<=3
condition2 = Agg_User_state['registered_Usage_count_std']>=-3
Agg_User_state = Agg_User_state[(condition1) & (condition2)]
print(Agg_User_state)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6853 entries, 0 to 6852
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   State              6853 non-null   object 
 1   Year               6853 non-null   object 
 2   Quater             6853 non-null   int64  
 3   Mobile_Brand_Used  6853 non-null   object 
 4   Usage_count        6853 non-null   int64  
 5   Percentage         6853 non-null   float64
dtypes: float64(1), int64(2), object(3)
memory usage: 321.4+ KB
     Mobile_Brand_Used  Usage_count  Percentage  encoded_Mobile_Brand_Used  \
0               Xiaomi         1665    0.247033                         19   
1              Samsung         1445    0.214392                         16   
2                 Vivo          982    0.145697                         18   
3                 Oppo          501    0.074332                         13   
4              OnePlus          332    0.049258         

In [10]:
#plotting aggregated_transaction_insurance in the form of india map using plotly
import plotly.express as px

fig = px.choropleth(
    Agg_User_state,
    geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
    featureidkey='properties.ST_NM',
    locations='State',
    color='Usage_count',
    color_continuous_scale='Reds'
)

fig.update_geos(fitbounds="locations", visible=False)

fig.show()

In [ ]:
#This is for aggregated no of users for the year 2018 data
import pandas as pd
import json
import os

#This is to direct the path to get the data as states

path=r"D:/BONEYS/WEB/PYTHON/Project/PhonePe/data/aggregated/user/country/india/2018"
Agg_user_2018_list=os.listdir(path)

#This is to extract the data's to create a dataframe
clm={'Quater':[],'Mobile_Brand_Used':[], 'Usage_count':[], 'Percentage':[]}

for i in Agg_user_2018_list:
    p_i=path+"/"+i
    Data=open(p_i,'r')
    D=json.load(Data)
    for z in D['data']['usersByDevice']:
        Brand=z['brand']
        count=z['count']
        perc=z['percentage']
        clm['Mobile_Brand_Used'].append(Brand)
        clm['Usage_count'].append(count)
        clm['Percentage'].append(perc)
        clm['Quater'].append(int(i.strip('.json')))
#Succesfully created a dataframe
Agg_users_2018=pd.DataFrame(clm)
print(Agg_users_2018)

    Quater Mobile_Brand_Used  Usage_count  Percentage
0        1            Xiaomi     11926334    0.254413
1        1           Samsung      9609401    0.204988
2        1              Vivo      5894293    0.125737
3        1              Oppo      4479351    0.095554
4        1            Realme      2376866    0.050703
5        1             Apple      1825153    0.038934
6        1          Motorola      1593250    0.033987
7        1           OnePlus      1536418    0.032775
8        1            Lenovo      1246507    0.026591
9        1            Huawei       808774    0.017253
10       1            Others      5581520    0.119065
11       2            Xiaomi     16416511    0.257925
12       2           Samsung     12943051    0.203353
13       2              Vivo      8365074    0.131426
14       2              Oppo      6246581    0.098142
15       2            Realme      3165738    0.049738
16       2             Apple      2326897    0.036559
17       2          Motorola